In [ ]:
#importamos las librerías necesarias
import json 
import pickle 
import zipfile 
import pandas as pd
!pip install tf_slim

     |████████████████████████████████| 352 kB 7.7 MB/s 


In [ ]:
#Descomprimimos el dataset original 
local_zip = "/content/Dataset-personas.zip"
zip_ref = zipfile.ZipFile(local_zip,"r")
zip_ref.extractall("dataset_original")
zip_ref.close()

In [ ]:
#Cargamos el archivo json
type_file = "train"
path = "/content/Deteccion de Personas - TRAIN.json"
data_file = open(path)
data = json.load(data_file)


In [ ]:
data

[{'classification': [],
  'height': '569',
  'id': '1c0972e9-6216-45db-b63e-f702294cab095aec6a3a-3f75-407c-ba13-3ad6480a07c7',
  'image': '1.png',
  'segmentation_url': '',
  'tags': [{'classes': [],
    'color': '#00c9c9',
    'id': 'b31c37dd-fa6f-4a2a-9b80-1b0150f9b193',
    'name': 'Personas',
    'parent': None,
    'pos': {'h': 513, 'w': 248, 'x': 154, 'y': 47},
    'text': None,
    'type': 'bounding_box'}],
  'width': '640'},
 {'classification': [],
  'height': '567',
  'id': '1c0972e9-6216-45db-b63e-f702294cab09ec5a98cd-a099-4720-8803-afc5851a24c5',
  'image': '2.png',
  'segmentation_url': '',
  'tags': [{'classes': [],
    'color': '#00c9c9',
    'id': '8d744127-2d6e-4434-ae22-5fc1d130e3fd',
    'name': 'Personas',
    'parent': None,
    'pos': {'h': 355, 'w': 255, 'x': 286.5, 'y': 32},
    'text': None,
    'type': 'bounding_box'}],
  'width': '573'},
 {'classification': [],
  'height': '535',
  'id': '1c0972e9-6216-45db-b63e-f702294cab09e9437262-bf9c-45f4-8857-511df8359878

In [ ]:
csv_list = []
"""Bucle for que recorrerá todas las imágenes y recogerá tanto tamaño (altura y 
anchura) como el nombre""" 
for classification in data:
  width,height = classification['width'], classification['height']
  image = classification['image']
  #Bucle For que recoge el nombre de la etiqueta y su posición
  for item in classification['tags']:
    name = item['name']
    xmin = item['pos']['x']
    ymin = item['pos']['y']
    xmax = item['pos']['x'] + item['pos']['w']
    ymax = item['pos']['y'] + item['pos']['h']

    '''
    Los datos recogidos serán guardados en una variable para después ir al 
    archivo CSV 
    ''' 

    value = (image,width,height,name,xmin,ymin,xmax,ymax)
    csv_list.append(value)

column_name = ['filename','width','height','class','xmin','ymin','xmax','ymax']
csv_df = pd.DataFrame(csv_list, columns = column_name)

csv_df.to_csv("/content/{}_labels.csv".format(type_file))

In [ ]:
#Proceso de instalación de la librería object detection
import os
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
%cd /content/models/
!git checkout 58d19c67e1d30d905dd5c6e5092348658fed80af
!apt-get update && apt-get install -y -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!python object_detection/builders/model_builder_test.py

/content
/content/models
Note: checking out '58d19c67e1d30d905dd5c6e5092348658fed80af'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 58d19c67e Internal change
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804

In [ ]:
#Trozo de codigo sacado de la documentación oficial de tensowflow 
"""
Usage:

# Create train data:
python generate_tfrecord.py --label=<LABEL> --csv_input=<PATH_TO_ANNOTATIONS_FOLDER>/train_labels.csv  --output_path=<PATH_TO_ANNOTATIONS_FOLDER>/train.record

# Create test data:
python generate_tfrecord.py --label=<LABEL> --csv_input=<PATH_TO_ANNOTATIONS_FOLDER>/test_labels.csv  --output_path=<PATH_TO_ANNOTATIONS_FOLDER>/test.record
"""

from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf
import sys
sys.path.append("../../models/research")

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict



"""Aquí ponemos el nombre de las etiquetas en este caso solo hay una que es 
  persona"""
def class_text_to_int(row_label):
    if row_label == "Personas":  # 'ship':
        return 1
    else: 
      None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    # check if the image format is matching with your images.
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example
#ponemos el nombre que tendrá el archivo
output_path = "train.record"
#ponemos la ruta de donde sacara las imagenes
image_dir = "/content/dataset_original/human detection dataset/1"
#ruta del archivo csv final
csv_input = "/content/train_labels.csv"

writer = tf.io.TFRecordWriter(output_path)
path = os.path.join(image_dir)
examples = pd.read_csv(csv_input)
grouped = split(examples, 'filename')
for group in grouped:
     tf_example = create_tf_example(group, path)
     writer.write(tf_example.SerializeToString())

writer.close()
output_path = os.path.join(os.getcwd(),output_path)
print('Successfully created the TFRecords: {}'.format(output_path))


Successfully created the TFRecords: /content/models/research/train.record
